In [1]:
import pandas as pd
import numpy as np
import time
import re
from tqdm import tqdm_notebook
from torrequest import TorRequest
from bs4 import BeautifulSoup
from retrying import retry
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ECimport 
import pandas as pd
import souper_v2
from tqdm import tqdm_notebook

In [2]:
sampled_players = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/players_transfermarket.csv')
sampled_players

,Unnamed: 0,link,player,team
0,0,/david-de-gea/profil/spieler/59377,David de Gea,Manchester United
1,1,/jack-butland/profil/spieler/128899,Jack Butland,Manchester United
2,2,/tom-heaton/profil/spieler/34130,Tom Heaton,Manchester United
3,3,/lisandro-martinez/profil/spieler/480762,Lisandro Martínez,Manchester United
4,4,/raphael-varane/profil/spieler/164770,Raphaël Varane,Manchester United
...,...,...,...,...
10749,22,/aidan-simmons/profil/spieler/867763,Aidan Simmons,Western Sydney Wanderers
10750,23,/kusini-yengi/profil/spieler/708099,Kusini Yengi,Western Sydney Wanderers
10751,24,/nathanael-blair/profil/spieler/1023268,Nathanael Blair,Western Sydney Wanderers
10752,25,/zachary-sapsford/profil/spieler/703657,Zachary Sapsford,Western Sydney Wanderers


In [3]:
def get_soup(URL):
    
    chrome_options = Options()
    chrome_options.add_argument('--window-size=1,1')
    
    #options = webdriver.ChromeOptions()
    #options.add_argument('--disable-popup-blocking')
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [4]:
# Hàm đưa giá trị chuyển nhượng từ kiểu string -> numeric
# Ví dụ '€30.00m' -> '30000000'
def clean_value(s):
    if s[0] == '€':
        s = s[1:]
    s1 = s[:-1]
    #s1 = s1.results.map(lambda x: str(x).replace('.',''))
    s2 = float(s1)
    if s[len(s)-1] == 'm':
        s2 = s2*1000000
    elif s[len(s)-1] == 'k':
        s2 = s2*1000
    return s2

In [5]:
# Thu thập giá trị chuyển nhượng của từng cầu thủ
def obtain_market_value(players_df):
     
    links = players_df['link']
    nationality = ""
    
    returner = {'link': [], 'current_value': [], 'highest_value': []}
    
    index = 0
    
    for each_link in links:
        
        # Obtaining final link
        new_link = 'https://www.transfermarkt.co.uk' + each_link
        print(new_link)
        soup = get_soup(new_link)
        
        # Obtaining highest and current value
        #values = [x.text.strip() for x in soup.find_all('div', attrs={'class':'tm-player-market-value-development__current-and-max'})]
        
        try:
            a = soup.find('div', class_ = "tm-player-market-value-development__current-value")
            current_value = a.text.strip()
            print(current_value)
            current_value = clean_value(current_value)
            
        except:
            current_value = 0
            
        try:
            highest_value = soup.find('div', class_ = "tm-player-market-value-development__max-value").text
            print(highest_value)
            highest_value = clean_value(highest_value)
        except:
            highest_value = 0
                    
        returner['current_value'].append(current_value)
        returner['highest_value'].append(highest_value)
        returner['link'].append(each_link)
        
        print("Finish", index)
        index += 1
                    
    return returner

## Obtaining data by parts

In [6]:
# Sample_data là một list, mỗi phần tử là 1 dataframe tách ra từ sampled_players
sample_data = []
sample_data.append(sampled_players.iloc[:10])
i = 10
while i < sampled_players.shape[0]:
    if i >= sampled_players.shape[0] - 200:
        sample_data.append(sampled_players.iloc[i:])
    else:
        sample_data.append(sampled_players.iloc[i:i+200])
    i += 200 # mỗi sample chứa 200 dòng
print(len(sample_data))

55


In [7]:
i = 3
sample = obtain_market_value(sample_data[i])
sample

https://www.transfermarkt.co.uk/dan-burn/profil/spieler/134270
€12.00m
€12.00m
Finish 0
https://www.transfermarkt.co.uk/jamal-lewis/profil/spieler/346018
€8.00m
€17.00m
Finish 1
https://www.transfermarkt.co.uk/paul-dummett/profil/spieler/170321
€1.20m
€7.00m
Finish 2
https://www.transfermarkt.co.uk/kieran-trippier/profil/spieler/95810
€13.00m
€35.00m
Finish 3
https://www.transfermarkt.co.uk/emil-krafth/profil/spieler/184528
€4.00m
€5.00m
Finish 4
https://www.transfermarkt.co.uk/javier-manquillo/profil/spieler/162029
€3.00m
€6.00m
Finish 5
https://www.transfermarkt.co.uk/harrison-ashby/profil/spieler/559130
€1.00m
€1.00m
Finish 6
https://www.transfermarkt.co.uk/bruno-guimaraes/profil/spieler/520624
€60.00m
€60.00m
Finish 7
https://www.transfermarkt.co.uk/joe-willock/profil/spieler/340329
€30.00m
€30.00m
Finish 8
https://www.transfermarkt.co.uk/sean-longstaff/profil/spieler/346707
€22.00m
€22.00m
Finish 9
https://www.transfermarkt.co.uk/matty-longstaff/profil/spieler/484387
€1.50m
€5.00m

€60.00m
€90.00m
Finish 87
https://www.transfermarkt.co.uk/arnaut-danjuma/profil/spieler/355861
€27.00m
€50.00m
Finish 88
https://www.transfermarkt.co.uk/dejan-kulusevski/profil/spieler/431755
€55.00m
€55.00m
Finish 89
https://www.transfermarkt.co.uk/lucas-moura/profil/spieler/77100
€9.00m
€40.00m
Finish 90
https://www.transfermarkt.co.uk/harry-kane/profil/spieler/132098
€90.00m
€150.00m
Finish 91
https://www.transfermarkt.co.uk/richarlison/profil/spieler/378710
€55.00m
€65.00m
Finish 92
https://www.transfermarkt.co.uk/edouard-mendy/profil/spieler/442531
€18.00m
€32.00m
Finish 93
https://www.transfermarkt.co.uk/kepa-arrizabalaga/profil/spieler/192279
€15.00m
€60.00m
Finish 94
https://www.transfermarkt.co.uk/gabriel-slonina/profil/spieler/656316
€8.00m
€8.00m
Finish 95
https://www.transfermarkt.co.uk/marcus-bettinelli/profil/spieler/116648
€2.00m
€4.00m
Finish 96
https://www.transfermarkt.co.uk/wesley-fofana/profil/spieler/475411
€65.00m
€65.00m
Finish 97
https://www.transfermarkt.co.uk/

€14.00m
€14.00m
Finish 174
https://www.transfermarkt.co.uk/omar-marmoush/profil/spieler/445939
€6.00m
€6.00m
Finish 175
https://www.transfermarkt.co.uk/dzenan-pejcinovic/profil/spieler/819636
€1.00m
€1.00m
Finish 176
https://www.transfermarkt.co.uk/oliver-baumann/profil/spieler/55089
€4.50m
€10.00m
Finish 177
https://www.transfermarkt.co.uk/philipp-pentke/profil/spieler/8246
€300k
€400k
Finish 178
https://www.transfermarkt.co.uk/luca-philipp/profil/spieler/432671
€300k
€300k
Finish 179
https://www.transfermarkt.co.uk/nahuel-noll/profil/spieler/600942
€250k
€250k
Finish 180
https://www.transfermarkt.co.uk/ozan-kabak/profil/spieler/361260
€10.00m
€32.00m
Finish 181
https://www.transfermarkt.co.uk/stanley-nsoki/profil/spieler/371141
€8.50m
€12.00m
Finish 182
https://www.transfermarkt.co.uk/kevin-akpoguma/profil/spieler/160241
€4.00m
€12.00m
Finish 183
https://www.transfermarkt.co.uk/kevin-vogt/profil/spieler/84435
€3.50m
€17.00m
Finish 184
https://www.transfermarkt.co.uk/john-anthony-broo

{'link': ['/dan-burn/profil/spieler/134270',
  '/jamal-lewis/profil/spieler/346018',
  '/paul-dummett/profil/spieler/170321',
  '/kieran-trippier/profil/spieler/95810',
  '/emil-krafth/profil/spieler/184528',
  '/javier-manquillo/profil/spieler/162029',
  '/harrison-ashby/profil/spieler/559130',
  '/bruno-guimaraes/profil/spieler/520624',
  '/joe-willock/profil/spieler/340329',
  '/sean-longstaff/profil/spieler/346707',
  '/matty-longstaff/profil/spieler/484387',
  '/matt-ritchie/profil/spieler/92469',
  '/joelinton/profil/spieler/333241',
  '/elliot-anderson/profil/spieler/567576',
  '/anthony-gordon/profil/spieler/503733',
  '/allan-saint-maximin/profil/spieler/272642',
  '/ryan-fraser/profil/spieler/146795',
  '/miguel-almiron/profil/spieler/272999',
  '/jacob-murphy/profil/spieler/199527',
  '/alexander-isak/profil/spieler/349066',
  '/callum-wilson/profil/spieler/123682',
  '/ederson/profil/spieler/238223',
  '/stefan-ortega/profil/spieler/85941',
  '/scott-carson/profil/spieler/1

In [8]:
df = pd.DataFrame(sample)
df

,link,current_value,highest_value
0,/dan-burn/profil/spieler/134270,12000000.0,12000000.0
1,/jamal-lewis/profil/spieler/346018,8000000.0,17000000.0
2,/paul-dummett/profil/spieler/170321,1200000.0,7000000.0
3,/kieran-trippier/profil/spieler/95810,13000000.0,35000000.0
4,/emil-krafth/profil/spieler/184528,4000000.0,5000000.0
...,...,...,...
195,/thomas-delaney/profil/spieler/91849,3000000.0,27000000.0
196,/finn-ole-becker/profil/spieler/334999,1000000.0,2500000.0
197,/umut-tohumcu/profil/spieler/569031,1000000.0,1000000.0
198,/robert-skov/profil/spieler/270393,9000000.0,14000000.0


In [9]:
file_name =  "C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtain_market_value/sample_data_" + str(i) + ".csv"
df.to_csv(file_name)

In [31]:
print(clean_value('10.00m'))

10000000.0
